<a href="https://colab.research.google.com/github/bobbercheng/ventilator-pressure-prediction/blob/master/ventilator_pressure_transformer_V6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

V1: Add PositionalEncoding

V2: Adjust Transformer parameters

V3: Use Transformer class instead of Tensorflow version

V4: Without EDA and only use 400 features that is very similar to Keras time serial transformer sample if features and time serial are flatten. TPU v2 supports 16 heads, 512 key_dim, 512 batch size. But converge is not good

V4: Max TPU V2 config, 128 heads, 512 key_dim, 512 batch size. Try build_complex_transfer_model. LP: 0.217, CX: 0.21, Train: 0.13. It looks complex transfer model can remember train data.

V5: Try to add decoder

V6: Complete transformer training. Please notice it only support one GUP/TPU
Batch size: 256. Bath size 512 crashes in Colab.

Epoch 10 Batch 0 Loss 0.8326
Epoch 10 Batch 50 Loss 1.1333
Epoch 10 Batch 100 Loss 1.1530
Epoch 10 Batch 150 Loss 1.1773
Epoch 10 Batch 200 Loss 1.1096
Epoch 10 Batch 250 Loss 1.0583
Epoch 10 Batch 0 Valid Loss 0.7480
Saving checkpoint for epoch 10 at /tmp/ckpt/train/transformer/ckpt-2
Epoch 10 Loss 1.0570 Valid Loss 0.7291
Time taken for 1 epoch: 2874.25 secs

Epoch 11 Batch 0 Loss 0.8168
Epoch 11 Batch 50 Loss 0.8682
Epoch 11 Batch 100 Loss 0.8439
Epoch 11 Batch 150 Loss 0.8419
Epoch 11 Batch 200 Loss 0.8387
Epoch 11 Batch 250 Loss 0.8345
Epoch 11 Batch 0 Valid Loss 0.6681
Epoch 11 Loss 0.8339 Valid Loss 0.6813
Time taken for 1 epoch: 2820.10 secs

Epoch 12 Batch 0 Loss 0.7892
Epoch 12 Batch 50 Loss 0.8301
Epoch 12 Batch 100 Loss 0.8450
Epoch 12 Batch 150 Loss 0.8319
Epoch 12 Batch 200 Loss 0.8287
Epoch 12 Batch 250 Loss 0.8282
Epoch 12 Batch 0 Valid Loss 0.6762
Epoch 12 Loss 0.8276 Valid Loss 0.6780
Time taken for 1 epoch: 2832.69 secs

Epoch 13 Batch 0 Loss 0.7996
Epoch 13 Batch 50 Loss 0.8137
Epoch 13 Batch 100 Loss 0.8002
Epoch 13 Batch 150 Loss 0.7959
Epoch 13 Batch 200 Loss 0.7964
Epoch 13 Batch 250 Loss 0.8010
Epoch 13 Batch 0 Valid Loss 0.6993
Epoch 13 Loss 0.8012 Valid Loss 0.7040
Time taken for 1 epoch: 2848.61 secs

Epoch 14 Batch 0 Loss 0.7913
Epoch 14 Batch 50 Loss 0.8400
Epoch 14 Batch 100 Loss 0.7992
Epoch 14 Batch 150 Loss 0.8004
Epoch 14 Batch 200 Loss 0.7989
Epoch 14 Batch 250 Loss 0.9859
Epoch 14 Batch 0 Valid Loss 3.7756
Epoch 14 Loss 1.0062 Valid Loss 3.5996
Time taken for 1 epoch: 2818.39 secs

Epoch 15 Batch 0 Loss 3.7839
Epoch 15 Batch 50 Loss 3.1948
Epoch 15 Batch 100 Loss 2.5220
Epoch 15 Batch 150 Loss 2.2678
Epoch 15 Batch 200 Loss 2.1025




In [1]:
# Update pandas version for Colab TPU
#!pip install pandas==1.3.2

In [2]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [3]:
if IN_COLAB:
  from google.colab import drive
  drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [4]:
if IN_COLAB:
  !pip install kaggle
  !mkdir /root/.kaggle
  !cp /gdrive/MyDrive/ventilator-pressure-prediction/kaggle.json /root/.kaggle
  !kaggle competitions download -c ventilator-pressure-prediction
  !mkdir -p /kaggle/input/ventilator-pressure-prediction
  !unzip '*.zip' -d /kaggle/input/ventilator-pressure-prediction
  !ls /kaggle/input/ventilator-pressure-prediction

mkdir: cannot create directory ‘/root/.kaggle’: File exists
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  sample_submission.csv.zip
replace /kaggle/input/ventilator-pressure-prediction/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N

Archive:  train.csv.zip

Archive:  test.csv.zip

3 archives were successfully processed.
sample_submission.csv  test.csv  train.csv


In [5]:
import numpy as np
import pandas as pd

# import optuna
import matplotlib.pyplot as plt
import gc
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

model_folder = '/gdrive/MyDrive/ventilator-pressure-prediction/transformer/'
checkpoints_folder = '/gdrive/MyDrive/ventilator-pressure-prediction/transformer/checkpoints/train'


In [6]:
import os
import random
# Deep Learning
import tensorflow as tf
from tensorflow import keras
# Metrics
from sklearn.metrics import mean_absolute_error

# Random Seed Initialize
RANDOM_SEED = 42

def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything()

In [7]:
DEBUG = False

train = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv')
submission = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/sample_submission.csv')

if DEBUG:
    train = train[:80*10000]

In [8]:
def add_features(df):
    df['cross']= df['u_in'] * df['u_out']
    df['cross2']= df['time_step'] * df['u_out']
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['time_step_cumsum'] = df.groupby(['breath_id'])['time_step'].cumsum()
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    print("Step-1...Completed")
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
    print("Step-2...Completed")
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    print("Step-3...Completed")
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    print("Step-4...Completed")
    
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] =df['u_in_cumsum'] /df['count']
    
    df['breath_id_lag']=df['breath_id'].shift(1).fillna(0)
    df['breath_id_lag2']=df['breath_id'].shift(2).fillna(0)
    df['breath_id_lagsame']=np.select([df['breath_id_lag']==df['breath_id']],[1],0)
    df['breath_id_lag2same']=np.select([df['breath_id_lag2']==df['breath_id']],[1],0)
    df['breath_id__u_in_lag'] = df['u_in'].shift(1).fillna(0)
    df['breath_id__u_in_lag'] = df['breath_id__u_in_lag'] * df['breath_id_lagsame']
    df['breath_id__u_in_lag2'] = df['u_in'].shift(2).fillna(0)
    df['breath_id__u_in_lag2'] = df['breath_id__u_in_lag2'] * df['breath_id_lag2same']
    print("Step-5...Completed")
    
    df['time_step_diff'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
    df['ewm_u_in_mean'] = (df\
                           .groupby('breath_id')['u_in']\
                           .ewm(halflife=9)\
                           .mean()\
                           .reset_index(level=0,drop=True))
    df[["15_in_sum","15_in_min","15_in_max","15_in_mean"]] = (df\
                                                              .groupby('breath_id')['u_in']\
                                                              .rolling(window=15,min_periods=1)\
                                                              .agg({"15_in_sum":"sum",
                                                                    "15_in_min":"min",
                                                                    "15_in_max":"max",
                                                                    "15_in_mean":"mean"
                                                                    #"15_in_std":"std"
                                                               })\
                                                               .reset_index(level=0,drop=True))
    print("Step-6...Completed")
    
    #df['u_in_diff_1_2'] = df['u_in_lag1'] - df['u_in_lag2']
    #df['u_out_diff_1_2'] = df['u_out_lag1'] - df['u_out_lag2']
    #df['u_in_lagback_diff_1_2'] = df['u_in_lag_back1'] - df['u_in_lag_back2']
    #df['u_out_lagback_diff_1_2'] = df['u_out_lag_back1'] - df['u_out_lag_back2']
    
    df['u_in_lagback_diff1'] = df['u_in'] - df['u_in_lag_back1']
    df['u_out_lagback_diff1'] = df['u_out'] - df['u_out_lag_back1']
    df['u_in_lagback_diff2'] = df['u_in'] - df['u_in_lag_back2']
    df['u_out_lagback_diff2'] = df['u_out'] - df['u_out_lag_back2']
    print("Step-7...Completed")
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    print("Step-8...Completed")
    
    return df

print("Train data...\n")
train = add_features(train)

print("\nTest data...\n")
test = add_features(test)



Train data...

Step-1...Completed
Step-2...Completed
Step-3...Completed
Step-4...Completed
Step-5...Completed
Step-6...Completed
Step-7...Completed
Step-8...Completed

Test data...

Step-1...Completed
Step-2...Completed
Step-3...Completed
Step-4...Completed
Step-5...Completed
Step-6...Completed
Step-7...Completed
Step-8...Completed


In [9]:
targets = train[['pressure']].to_numpy().reshape(-1, 80)

u_outs = train[['u_out']].to_numpy().reshape(-1, 80)

train.drop(['pressure','id', 'breath_id'], axis=1, inplace=True)

test = test.drop(['id', 'breath_id'], axis=1)

train.head(), train.shape

(   time_step       u_in  u_out  ...  R__C_5__10  R__C_5__20  R__C_5__50
 0   0.000000   0.083334      0  ...           0           0           0
 1   0.033652  18.383041      0  ...           0           0           0
 2   0.067514  22.509278      0  ...           0           0           0
 3   0.101542  22.808822      0  ...           0           0           0
 4   0.135756  25.355850      0  ...           0           0           0
 
 [5 rows x 70 columns], (6036000, 70))

In [10]:
train.drop(['one','count',
            'breath_id_lag','breath_id_lag2','breath_id_lagsame',
            'breath_id_lag2same'], axis=1, inplace=True)

test = test.drop(['one','count','breath_id_lag',
                  'breath_id_lag2','breath_id_lagsame',
                  'breath_id_lag2same'], axis=1)

In [11]:
RS = RobustScaler()
train = RS.fit_transform(train)
test = RS.transform(test)

In [12]:
train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

In [13]:
from tensorflow.keras import layers
from tensorflow import keras

In [14]:
# refer to https://www.tensorflow.org/text/tutorials/transformer#encoder
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, **kwargs):
    super(MultiHeadAttention, self).__init__(**kwargs)
    self.num_heads = num_heads
    self.d_model = d_model
    assert self.d_model % self.num_heads == 0
    self.depth = d_model // self.num_heads
    self.build_components()

  def build_components(self):
    self.wq = tf.keras.layers.Dense(self.d_model)
    self.wk = tf.keras.layers.Dense(self.d_model)
    self.wv = tf.keras.layers.Dense(self.d_model)

    self.dense = tf.keras.layers.Dense(self.d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

  # def get_config(self):
  #   config = super(MultiHeadAttention, self).get_config()
  #   config.update({"num_heads": self.num_heads,
  #           "d_model": self.d_model})
  #   return config

def point_wise_feed_forward_network(d_model, dff):
  # Feed forward can be Den lay or CNN as long as it keep final dimension
  #x = layers.Conv1D(filters=ff_dims, kernel_size=1, activation="selu")(x)
  #x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1, **kwargs):
    super(EncoderLayer, self).__init__(**kwargs)
    self.d_model = d_model
    self.num_heads = num_heads
    self.dff = dff
    self.rate = rate
    self.build_components()

  def build_components(self):
    self.mha = MultiHeadAttention(self.d_model, self.num_heads)
    self.ffn = point_wise_feed_forward_network(self.d_model, self.dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(self.rate)
    self.dropout2 = tf.keras.layers.Dropout(self.rate)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

  # def get_config(self):
  #   config = super(EncoderLayer, self).get_config()
  #   config.update({"num_heads": self.num_heads,
  #           "d_model": self.d_model,
  #           'dff': self.dff,
  #           'rate': self.rate
  #           })
  #   return config

class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, 
               #input_vocab_size,
               maximum_position_encoding, rate=0.1, **kwargs):
    super(Encoder, self).__init__(**kwargs)

    self.num_layers = num_layers
    self.d_model = d_model
    self.num_heads = num_heads
    self.dff = dff
    self.rate = rate
    self.maximum_position_encoding = maximum_position_encoding
    self.build_components()

  def build_components(self):
    # self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.embedding = tf.keras.layers.Dense(self.d_model)
    self.pos_encoding = positional_encoding(self.maximum_position_encoding,
                                            self.d_model)

    self.enc_layers = [EncoderLayer(self.d_model, self.num_heads, self.dff, self.rate)
                       for _ in range(self.num_layers)]

    self.dropout = tf.keras.layers.Dropout(self.rate)

  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]

    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x  # (batch_size, input_seq_len, d_model)

  # def get_config(self):
  #   config = super(Encoder, self).get_config()
  #   config.update({'num_layers': self.num_layers,
  #           "num_heads": self.num_heads,
  #           "d_model": self.d_model,
  #           'dff': self.dff,
  #           'rate': self.rate,
  #           'maximum_position_encoding': self.maximum_position_encoding
  #           })
  #   return config

class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1, **kwargs):
    super(DecoderLayer, self).__init__(**kwargs)
    self.d_model = d_model
    self.num_heads = num_heads
    self.dff = dff
    self.rate = rate
    self.build_components()

  def build_components(self):
    self.mha1 = MultiHeadAttention(self.d_model, self.num_heads)
    self.mha2 = MultiHeadAttention(self.d_model, self.num_heads)

    self.ffn = point_wise_feed_forward_network(self.d_model, self.dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(self.rate)
    self.dropout2 = tf.keras.layers.Dropout(self.rate)
    self.dropout3 = tf.keras.layers.Dropout(self.rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

    return out3, attn_weights_block1, attn_weights_block2

  # def get_config(self):
  #   config = super(DecoderLayer, self).get_config()
  #   config.update({"num_heads": self.num_heads,
  #           "d_model": self.d_model,
  #           'dff': self.dff,
  #           'rate': self.rate
  #           })
  #   return config



class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, 
              #  target_vocab_size,
               maximum_position_encoding, rate=0.1, **kwargs):
    super(Decoder, self).__init__(**kwargs)

    self.d_model = d_model
    self.num_layers = num_layers
    self.num_heads = num_heads
    self.dff = dff
    self.rate = rate
    self.maximum_position_encoding = maximum_position_encoding
    self.build_components()

  def build_components(self):
    # self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.embedding = tf.keras.layers.Dense(self.d_model)
    self.pos_encoding = positional_encoding(self.maximum_position_encoding, self.d_model)

    self.dec_layers = [DecoderLayer(self.d_model, self.num_heads, self.dff, self.rate)
                       for _ in range(self.num_layers)]
    self.dropout = tf.keras.layers.Dropout(self.rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

  # def get_config(self):
  #   config = super(Decoder, self).get_config()
  #   config.update({'num_layers': self.num_layers,
  #           "num_heads": self.num_heads,
  #           "d_model": self.d_model,
  #           'dff': self.dff,
  #           'rate': self.rate,
  #           'maximum_position_encoding': self.maximum_position_encoding
  #           })
  #   return config

class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, 
              #  input_vocab_size, target_vocab_size,
               pe_input, pe_target, rate=0.1, target_dim=1, **kwargs):
    super().__init__(**kwargs)
    self.d_model = d_model
    self.num_layers = num_layers
    self.num_heads = num_heads
    self.dff = dff
    self.rate = rate
    self.pe_input = pe_input
    self.pe_target = pe_target
    self.target_dim = target_dim
    self.build_components()

  def build_components(self):
    self.encoder = Encoder(self.num_layers, self.d_model, self.num_heads, self.dff,
                            #  input_vocab_size, 
                           self.pe_input, self.rate)

    self.decoder = Decoder(self.num_layers, self.d_model, self.num_heads, self.dff,
                          #  target_vocab_size, 
                           self.pe_target, self.rate)

    self.final_layer = tf.keras.layers.Dense(self.target_dim)

  def call(self, inputs, training):
    # Keras models prefer if you pass all your inputs in the first argument
    inp, tar = inputs

    enc_padding_mask, look_ahead_mask, dec_padding_mask = self.create_masks(inp, tar)

    enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)

    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

    return final_output, attention_weights

  def create_masks(self, inp, tar):
    # Encoder padding mask
    # enc_padding_mask = create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    # dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    # dec_target_padding_mask = create_padding_mask(tar)
    # look_ahead_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    # return enc_padding_mask, look_ahead_mask, dec_padding_mask
    return None, look_ahead_mask, None

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000, **kwargs):
    super(CustomSchedule, self).__init__(**kwargs)

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

n, d = 80, 64
pos_encoding = positional_encoding(n, d)
print(pos_encoding.shape)
pos_encoding = pos_encoding[0]

# Juggle the dimensions for the plot
pos_encoding = tf.reshape(pos_encoding, (n, d//2, 2))
pos_encoding = tf.transpose(pos_encoding, (2, 1, 0))
pos_encoding = tf.reshape(pos_encoding, (d, n))

plt.pcolormesh(pos_encoding, cmap='RdBu')
plt.ylabel('Depth')
plt.xlabel('Position')
plt.colorbar()
plt.show()

In [15]:
# **** Can not print anything if loss_function is called by train_step
def loss_function(tar_real_u_outs, pred, cols = 80):
  real = tar_real_u_outs[:, :cols, :]
  u_out = tar_real_u_outs[:, cols:, :]
  loss_ = loss_object(real, pred)

  mask = 1 - u_out
  mask = tf.squeeze(mask, [-1])
  loss_ *= mask

  #If all are masked, return 0
  # if tf.math.count_nonzero(mask) == 0:
  #   return 0.0
  
  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

def accuracy_function(real, pred):
  accuracies = tf.equal(real, pred)
  accuracies = tf.cast(accuracies, dtype=tf.float32)
  # print(accuracies)
  return tf.reduce_mean(accuracies)

temp_mha = MultiHeadAttention(d_model=512, num_heads=8)
y = tf.random.uniform((1, 60, 512))  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y, k=y, q=y, mask=None)
out.shape, attn.shape

sample_ffn = point_wise_feed_forward_network(512, 2048)
sample_ffn(tf.random.uniform((64, 50, 512))).shape

sample_encoder_layer = EncoderLayer(512, 8, 2048)

sample_encoder_layer_output = sample_encoder_layer(
    tf.random.uniform((64, 43, 512)), False, None)

sample_encoder_layer_output.shape  # (batch_size, input_seq_len, d_model)

sample_decoder_layer = DecoderLayer(512, 8, 2048)
config = sample_decoder_layer.get_config()
print(config)
new_layer = DecoderLayer.from_config(config)

sample_decoder_layer_output, _, _ = new_layer(
    tf.random.uniform((64, 50, 512)), sample_encoder_layer_output,
    False, None, None)

sample_decoder_layer_output.shape  # (batch_size, target_seq_len, d_model)


sample_encoder = Encoder(num_layers=2, d_model=512, num_heads=8,
                         dff=2048, 
                        #  input_vocab_size=8500,
                         maximum_position_encoding=10000)
temp_input = tf.random.uniform((64, 62, 512), dtype=tf.int64, minval=0, maxval=200)

sample_encoder_output = sample_encoder(temp_input, training=False, mask=None)

print(sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)

sample_decoder = Decoder(num_layers=2, d_model=512, num_heads=8,
                         dff=2048, 
                        #  target_vocab_size=8000,
                         maximum_position_encoding=5000)
temp_input = tf.random.uniform((64, 26, 512), dtype=tf.int64, minval=0, maxval=200)

output, attn = sample_decoder(temp_input,
                              enc_output=sample_encoder_output,
                              training=False,
                              look_ahead_mask=None,
                              padding_mask=None)

output.shape, attn['decoder_layer2_block2'].shape


d_model = 512
temp_learning_rate_schedule = CustomSchedule(d_model)

plt.plot(temp_learning_rate_schedule(tf.range(40000, dtype=tf.float32)))
plt.ylabel("Learning Rate")
plt.xlabel("Train Step")

sample_transformer = Transformer(
    num_layers=2, d_model=512, num_heads=8, dff=2048,
    # input_vocab_size=8500, target_vocab_size=8000,
    pe_input=10000, pe_target=6000)

temp_input = tf.random.uniform((64, 38, 512), dtype=tf.int64, minval=0, maxval=200)
temp_target = tf.random.uniform((64, 36, 512), dtype=tf.int64, minval=0, maxval=200)

fn_out, _ = sample_transformer([temp_input, temp_target], training=False)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)

In [16]:
num_layers=6
d_model = 512
dff = 2048
num_heads = 8
dropout_rate = 0.1

In [17]:
START = -2
def add_Start_to_targets(targets, u_outs):
  new_targets = np.insert(np.append(targets, u_outs, axis=1), 0, START, axis=1).reshape(-1, targets.shape[1] + u_outs.shape[-1] + 1, 1)
  return new_targets

def get_targets_without_Start(targets):
  return targets[:, 1:]

new_targets = add_Start_to_targets(targets)
new_targets.shape, targets.shape

In [18]:
def getOptimizer():
  learning_rate = CustomSchedule(d_model)

  optimizer = tf.keras.optimizers.Adam(
      learning_rate, 
      beta_1=0.9, beta_2=0.98,epsilon=1e-9)
  return optimizer

optimizer = getOptimizer()

In [19]:
loss_object = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.NONE)

In [20]:
y_true = np.zeros([2, 160, 1])
y_true[-1, 58, 0] = 1
y_true[-1, -1, 0] = 1
y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
# pred and u_out
y_pred = tf.ones([2, 80, 1], tf.float32)
test_loss = loss_function(y_true, y_pred)
print('loss:', test_loss)

assert test_loss == 158/159

loss: tf.Tensor(0.9937107, shape=(), dtype=float32)


y_true = tf.constant([[[0.], [1.]], [[0.], [0.]]])
y_pred = tf.constant([[[2.], [1.]], [[2.], [0.]]])
y_true.shape, accuracy_function(y_true, y_pred)

In [21]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
valid_loss = tf.keras.metrics.Mean(name='valid_loss')
# train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

In [22]:
def get_transformer():
  transformer = Transformer(
      num_layers=num_layers,
      d_model=d_model,
      num_heads=num_heads,
      dff=dff,
      # input_vocab_size=tokenizers.pt.get_vocab_size().numpy(),
      # target_vocab_size=tokenizers.en.get_vocab_size().numpy(),
      pe_input=80,
      pe_target=80,
      rate=dropout_rate)
  return transformer

transformer = get_transformer()

In [23]:
checkpoint_path = "/tmp/ckpt/train/transformer"

#copy latest checkpoint from google driver
!mkdir -p /tmp/ckpt/train/transformer
!cp /gdrive/MyDrive/ventilator-pressure-prediction/transformer/checkpoints/train/* /tmp/ckpt/train/transformer

In [24]:
if IN_COLAB:
  save_options = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint, options=save_options)
  print('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [25]:
EPOCHS = 100
BATCH_SIZE = 256#1024
NUM_FOLDS = 7

# Function to get hardware strategy
def get_hardware_strategy():
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        tf.config.optimizer.set_jit(True)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    return tpu, strategy

tpu, strategy = get_hardware_strategy()

In [26]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

train_step_signature = [
    tf.TensorSpec(shape=(None, 80, 64), dtype=tf.float32),
    tf.TensorSpec(shape=(None, 161, 1), dtype=tf.float32),
]


@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :80, :]
  tar_real = tar[:, 1:81, :]
  tar_real_u_outs = tar[:, 1:, :]
  # print(inp.shape, tar_inp.shape, tar_real.shape, tar_real_u_outs.shape)

  with tf.GradientTape() as tape:
    predictions, _ = transformer([inp, tar_inp],
                                 training = True)
    loss = loss_function(tar_real_u_outs, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

  train_loss(loss)
  # train_accuracy(accuracy_function(tar_real, predictions))

train_step_signature = [
    tf.TensorSpec(shape=(None, 80, 64), dtype=tf.float32),
    tf.TensorSpec(shape=(None, 161, 1), dtype=tf.float32),
]

valid_step_signature = [
    tf.TensorSpec(shape=(None, 80, 64), dtype=tf.float32),
    tf.TensorSpec(shape=(None, 161, 1), dtype=tf.float32),
]
@tf.function(input_signature=valid_step_signature)
def valid_step(inp, tar):
  tar_inp = tar[:, :80, :]
  tar_real = tar[:, 1:81, :]
  tar_real_u_outs = tar[:, 1:, :]
  # print(inp.shape, tar_inp.shape, tar_real.shape, tar_real_u_outs.shape)

  predictions, _ = transformer([inp, tar_inp],
                                training = False)
  loss = loss_function(tar_real_u_outs, predictions)

  valid_loss(loss)
  return loss

In [ ]:
def make_batches(ds):
  return (
      ds
      .cache()
      # .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      # .map(tokenize_pairs, num_parallel_calls=tf.data.AUTOTUNE)
      .prefetch(tf.data.AUTOTUNE))

# Please don't use multiple GUP or TPU for train here
# with strategy.scope():

kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=2021)
test_preds = []
train_preds = []
for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
  print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
  X_train, X_valid = train[train_idx], train[test_idx]
  y_train, y_valid = targets[train_idx], targets[test_idx]
  u_out_train, u_out_valid = u_outs[train_idx], u_outs[test_idx]

  train_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(X_train, dtype=tf.float32), tf.cast(add_Start_to_targets(y_train, u_out_train), dtype=tf.float32)))
  valid_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(X_valid, dtype=tf.float32), tf.cast(add_Start_to_targets(y_valid, u_out_valid), dtype=tf.float32)))

  train_batches = make_batches(train_dataset)
  val_batches = make_batches(valid_dataset)

  for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
    valid_loss.reset_states()
    # train_accuracy.reset_states()

    # Train batch loop
    for (batch, (inp, tar)) in enumerate(train_batches):
      # print(inp.shape, tar.shape)
      train_step(inp, tar)

      if batch % 50 == 0:
        current_lr = optimizer._decayed_lr('float32').numpy()
        print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Learning rate {current_lr:.4f}')
      # else:
      #   print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f}')

    # Valid batch loop
    for (batch, (inp, tar)) in enumerate(val_batches):
      # print(inp.shape, tar.shape)
      valid_step(inp, tar)

      if batch % 50 == 0:
        print(f'Epoch {epoch + 1} Batch {batch} Valid Loss {valid_loss.result():.4f}')
    

    if (epoch + 1) % 2 == 0:
      ckpt_save_path = ckpt_manager.save(options=save_options)
      print(f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')
      if IN_COLAB:
        !cp /tmp/ckpt/train/transformer/* /gdrive/MyDrive/ventilator-pressure-prediction/transformer/checkpoints/train

    print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Valid Loss {valid_loss.result():.4f}')

    print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')
  break


--------------- > Fold 1 < ---------------
Epoch 1 Batch 0 Loss 0.8203 Learning rate 0.0004
Epoch 1 Batch 50 Loss 0.8344 Learning rate 0.0005
Epoch 1 Batch 100 Loss 0.8396 Learning rate 0.0005
Epoch 1 Batch 150 Loss 0.8372 Learning rate 0.0005
Epoch 1 Batch 200 Loss 0.8337 Learning rate 0.0005
Epoch 1 Batch 250 Loss 0.8365 Learning rate 0.0005
Epoch 1 Batch 0 Valid Loss 0.8874
Epoch 1 Loss 0.8365 Valid Loss 0.8513
Time taken for 1 epoch: 2839.91 secs

Epoch 2 Batch 0 Loss 0.8566 Learning rate 0.0005
Epoch 2 Batch 50 Loss 0.8649 Learning rate 0.0005
Epoch 2 Batch 100 Loss 0.8235 Learning rate 0.0005
Epoch 2 Batch 150 Loss 0.8288 Learning rate 0.0005
Epoch 2 Batch 200 Loss 0.8264 Learning rate 0.0005
Epoch 2 Batch 250 Loss 1.6590 Learning rate 0.0005
Epoch 2 Batch 0 Valid Loss 4.7205
Saving checkpoint for epoch 2 at /tmp/ckpt/train/transformer/ckpt-3
Epoch 2 Loss 1.6814 Valid Loss 4.5223
Time taken for 1 epoch: 2843.56 secs

Epoch 3 Batch 0 Loss 4.7976 Learning rate 0.0005
Epoch 3 Batch 

In [ ]:
print(optimizer._decayed_lr('float32').numpy())

In [ ]:
#reset Keras Session
def reset_keras():
    sess = tf.compat.v1.keras.backend.get_session()
    tf.compat.v1.keras.backend.clear_session()
    sess.close()
    sess = tf.compat.v1.keras.backend.get_session()

    # use the same config as you used to create the session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))
    gc.collect()

reset_keras()

print('Train MAE:', mae(targets.flatten(), np.median(np.vstack(train_preds),axis=0)))
PRESSURE_STEP = 0.07030214545120961
PRESSURE_MIN = -1.8957442945646408
PRESSURE_MAX = 64.82099173863948
# ENSEMBLE FOLDS WITH MEAN
submission["pressure"] = sum(test_preds)/len(test_preds)
submission.to_csv('submission_mean.csv', index=False)
# ENSEMBLE FOLDS WITH MEDIAN
submission["pressure"] = np.median(np.vstack(test_preds),axis=0)
submission.to_csv('submission_median.csv', index=False)
# ENSEMBLE FOLDS WITH MEDIAN AND ROUND PREDICTIONS
submission["pressure"] =\
    np.round( (submission.pressure - PRESSURE_MIN)/PRESSURE_STEP ) * PRESSURE_STEP + PRESSURE_MIN
submission.pressure = np.clip(submission.pressure, PRESSURE_MIN, PRESSURE_MAX)
submission.to_csv('submission_median_round.csv', index=False)

if IN_COLAB:
  !zip submission.zip submission_median_round.csv
  with open(model_folder + 'submission.zip', 'wb') as f:
    f.write(open('submission.zip', 'rb').read())
  # !zip cnn_models.zip *.hdf5
  # with open(model_folder + 'cnn_models.zip', 'wb') as f:
  #   f.write(open('cnn_models.zip', 'rb').read())

!cp /tmp/ckpt/train/transformer/* /gdrive/MyDrive/ventilator-pressure-prediction/transformer/checkpoints/train